# Lab3: Reviewing Product Inventory in a Cloud SQL Database

**Tutorial Prerequisites**
- Basic understanding of Python

## What is a Database

A _database_, at its core, is an organized collection of data. Databases are meant to be fast at retrieving, storing, and updating these collections. To help keep things organized, a database keeps related data in a collection called a _table_. Data is stored inside a table as _columns_ and _rows_. The _columns_ identify the data that is being stored in the table, and the _rows_ are the actual data itself. Refer to the figure below for a basic example of organized data inside the database, with rows and columns.

| name | city | code |
|-------|-------|-------|
| Detroit Metropolitan Airport | Romulous | DTW |
| Seattle-Tacoma International Airport | SeaTac | SEA |
| Los Angeles International Airport | Los Angeles | LAX |

## Moving Inventory Data from a Spreadsheet into a Database

Now the fun stuff begins, and you have a chance to get your hands dirty.
In the following you will:
- Provision (and allow connections to) a MySQL database instance.
- Connect to the MySQL instance using python.
- Execute SQL within python and print the result.
- Insert data from a CSV file into the database using python.

### Provisioning a Database
In this lab we will be importing our sample inventory data, and order history from a CSV into a SQL Database. To begin, make sure you are signed into your Google account and can access the [Google Cloud Console](https://console.cloud.google.com/).

Now that you're inside the Google Cloud Platform (commonly referred to a _GCP_), let's start putting together our database!

From the search bar at the top of the page, type in `SQL`. This will filter out the _many_ Google Cloud products. From the dropdown list, select **SQL** under **PRODUCTS & PAGES**

![Select the SQL item from the dropdown](assets/lab3/select_sql.png)

Click the **CREATE INSTANCE** button

![create instance button](assets/lab3/create_instance_button.png)

and then select **MySQL** from the options.

![select the mysql option](assets/lab3/create_mysql.png)


#### Setting the MySQL Database Configuation

The database creation wizard is displayed below. In your browser, fill out the following items:

- **Instance ID**: This is permanent and cannot be changed. Make it something descriptive.
- **Root Password**: This is the password we'll use in our python code to connect to the database. You also have the option to have Google generate a random, complex password for you. Don't lose this!
- **Region**: Which datacenter your database will be located. Select `us-central1` from the dropdown.
- **Database version**: The versions of MySQL that GCP has available. For our purposes, version 5.7 is fine.

![database config info](assets/lab3/database_setup.png)

Click the **CREATE** button!
The database will take some time to provision. Grab a coffee, but make sure its a quick one, you don't have enough time to grind the beans yourself.

#### Exposing Connections

Once the database and provisioned, click on the name from the list. This gives you a 10,000 foot view of how your database is operating, and some key pieces of information is presented (i.e. public IP address). Even though you can see your database running, you cannot connect to it. Google Cloud's firewall rules do not allow for incomming connections through third-party apps. If you have the `gcloud` command installed on your machine, you can connect to it through the [Cloud SQL Proxy](https://cloud.google.com/sql/docs/mysql/sql-proxy).
This is not ideal.
Our python application needs to be able to connect to our database, from any user, anywhere in the world.
So let's fix that!

On the lefthand side of the screen, click **Connections**.
When the page has loaded, the first section controls the connectivity of the database and should look similar to the image below.

![database connectivity](assets/lab3/connectivity.png)

Click the **Add network** button and enter the following:
- **Name**: The Internet
- **Network**: `0.0.0.0/0`

This configuration allows any connection from any IP address, and it is ***not*** secure for a production enviornment.
In fact, GCP will even show you a warning message about how this is a bad idea. Ignore this and click **Done** and then **Save**.

### Connect to a database using Python

Edit the code below with information about your MySQL database. You will be connecting with the `root` user, but the connection IP address and password you will need to fill in yourself.

In [10]:
! pip install mysql-connector

import mysql.connector
## Make sure the Host matches your GCP instance along with the password created earlier.
db = mysql.connector.connect(
  host="34.94.61.132",
  user="root",
  passwd="superadmin123"
)

cursor = db.cursor()
cursor.execute("SELECT VERSION() FROM DUAL")
print(cursor.fetchone()[0])

# clean up
cursor.close()
db.close()

5.7.25-google-log


When you run the code above, you'll see the current version of your MySQL instance. This is a good indicator that your database is up and running and that you can connect to it with python. Now let's prep MySQL for the data.

### Creating the Database and Schema

We must create the table within the database which will hold our inventory data.
The table will be called `product_table` which is defined by a _schema_ which contains the various columns along with datatypes.
Our data schema looks like the following.
Don't be alarmed if you don't understand all the details.
Just note that the fields in the table are a representation of the headers in the CSV file.

```sql
product_table (
    ProductName VARCHAR(50),
    Description VARCHAR(255),
    ProductSku BIGINT(12),
    QuantityInStock INT(5),
    ReorderLevel INT(5),
    WholeSaleCost DECIMAL(8,2),
    RetailCost DECIMAL(8,2)
)
```

To do create this in python, execute the following code.

In [33]:
! pip install mysql-connector

import mysql.connector
# make sure the host matches your GCP instance along with the password created earlier.
db = mysql.connector.connect(
  host="34.94.61.132",
  user="root",
  passwd="superadmin123"
)

cursor = db.cursor()

# always start fresh. remove the database if it exists
cursor.execute("DROP DATABASE IF EXISTS inventory")

# create the database within MySQL
cursor.execute("CREATE DATABASE IF NOT EXISTS inventory")

# tell MySQL to use the database and any operations are applied to it
cursor.execute("USE inventory")

# create the schema described above
cursor.execute("CREATE TABLE IF NOT EXISTS product_table (ProductName varchar(50),Description varchar(255),ProductSku bigint(12),QuantityInStock int(5),ReorderLevel int(5),WholeSaleCost decimal(8,2),RetailCost decimal(8,2));")

# confirm the table was successfully created
cursor.execute("SHOW TABLES")
print('tables:', [t[0] for t in cursor.fetchall()])

# clean up
cursor.close()
db.close()

tables: ['product_table']


### Insert Data From CSV into the Database

Now we will import our data from the provided `Lab3_inventory.csv` into our newly created table.  The code below iterates through the csv provided, and inserts every row into our table.

In [35]:
! pip install mysql-connector

import csv
import mysql.connector
# make sure the host matches your GCP instance along with the password created earlier.
db = mysql.connector.connect(
  host="34.94.61.132",
  user="root",
  passwd="superadmin123",
  database='inventory'
)

cursor = db.cursor()

with open('./data_files/lab3/inventory.csv') as datafile:
    csv_data = csv.reader(datafile)
    next(csv_data) 
    for row in csv_data:
        print('Inserting row into database:', row)
        cursor.execute ("INSERT INTO product_table (ProductName,Description,ProductSku,QuantityInStock,ReorderLevel,WholeSaleCost,RetailCost) VALUES (%s, %s, %s, %s, %s, %s, %s)",row)
    print('Committing changes to database')
    db.commit()
    
print('Success!')

Inserting row into database: ['Mens Shampoo', 'Best brand Mens Shampoo', '1034403303', '10', '3', '3.25', '6.99']
Inserting row into database: ['ScrewDriver Toolset', '3 Phillips, 2 Flathead toolset', '103443101', '4', '4', '8', '12.99']
Inserting row into database: ['KeepFresh Detergent', 'Original detegent scent for washing clothes', '1034411101', '15', '3', '8.25', '13.99']
Inserting row into database: ['MouthWash', 'Fresh Mint Mout wash', '122033020', '1', '5', '3.14', '4.99']
Committing changes to database
Success!


Congratulations! The database is now populated with data!

#### **Lab:** Selecting data from a Cloud SQL Database
Using the code from above as a guide, try to view all the data in the database.
The SQL statement to use is `SELECT * FROM product_table`.
Try and complete the following code.

In [43]:
! pip install mysql-connector

import mysql.connector

# connect to the database

# retreive a cursor from the database

# execute the SQL command in the cursor

# fetchall() the data from the cursor into a variable called results

# for row in results:
#   print(row)

# close() the cursor

# close() the database

('Mens Shampoo', 'Best brand Mens Shampoo', 1034403303, 10, 3, Decimal('3.25'), Decimal('6.99'))
('ScrewDriver Toolset', '3 Phillips, 2 Flathead toolset', 103443101, 4, 4, Decimal('8.00'), Decimal('12.99'))
('KeepFresh Detergent', 'Original detegent scent for washing clothes', 1034411101, 15, 3, Decimal('8.25'), Decimal('13.99'))
('MouthWash', 'Fresh Mint Mout wash', 122033020, 1, 5, Decimal('3.14'), Decimal('4.99'))
('Sample Product', 'Sample Description', 123456, 4, 2, Decimal('3.55'), Decimal('9.99'))
('Sample Product', 'Sample Description', 123456, 4, 2, Decimal('3.55'), Decimal('9.99'))


The output should resemble the following:
```
('Mens Shampoo', 'Best brand Mens Shampoo', 1034403303, 10, 3, Decimal('3.25'), Decimal('6.99'))
('ScrewDriver Toolset', '3 Phillips, 2 Flathead toolset', 103443101, 4, 4, Decimal('8.00'), Decimal('12.99'))
('KeepFresh Detergent', 'Original detegent scent for washing clothes', 1034411101, 15, 3, Decimal('8.25'), Decimal('13.99'))
('MouthWash', 'Fresh Mint Mout wash', 122033020, 1, 5, Decimal('3.14'), Decimal('4.99'))
```

#### **Example**: Getting a Python Dictionary of Product Names and Prices

In [44]:
! pip install mysql-connector

import mysql.connector

# make sure the host matches your GCP instance along with the password created earlier.
db = mysql.connector.connect(
  host='34.94.61.132',
  user='root',
  passwd='superadmin123',
  database='inventory'
)

cursor = db.cursor()

cursor.execute("SELECT ProductName, RetailCost FROM product_table")

products = [{'name':rslt[0], 'price':rslt[1]} for rslt in cursor.fetchall()]

print('product dictionary:', products)

for product in products:
    print('{:25}{:05.2f}'.format(product['name'], product['price']))

# clean up
cursor.close()
db.close()

product dictionary: [{'name': 'Mens Shampoo', 'price': Decimal('6.99')}, {'name': 'ScrewDriver Toolset', 'price': Decimal('12.99')}, {'name': 'KeepFresh Detergent', 'price': Decimal('13.99')}, {'name': 'MouthWash', 'price': Decimal('4.99')}]
Mens Shampoo             06.99
ScrewDriver Toolset      12.99
KeepFresh Detergent      13.99
MouthWash                04.99


#### **Lab:** Add your own data into the table!
Using the SQL Insert Statement add a couple rows into the table, and then display the result. Example: `INSERT INTO product_table (ProductName,Description,ProductSku,QuantityInStock,ReorderLevel,WholeSaleCost,RetailCost) VALUES ('Sample Product', 'Sample Description', 123456, 4, 2, 3.55, 9.99)`

In [ ]:
! pip install mysql-connector
import mysql.connector
import csv

db = mysql.connector.connect(
  host="34.94.61.132",
  user="root",
  passwd="superadmin123",
  database='inventory'
)

cursor = db.cursor()
## Add your own execute Insert statement here!    

db.commit()

## Prints out the rows in the table
cursor.execute("SELECT * FROM product_table") 

result = cursor.fetchall()
print(result)

#### **Lab:** Try querying our database to find products that have a retail cost more than $6.

A query is a question we ask the database about our data. The language of retrieving our data is call _Structured Query Language_ or _SQL_.  
To ask our database a question we must do so in the format `SELECT * FROM product_table WHERE WholeSaleCost > 7` The field `WholeSaleCost` can be any colume in our database.  Check out [SQL Operators from W3](https://www.w3schools.com/sql/sql_operators.asp) to understand the questions we can ask about our data.

In [ ]:
! pip install mysql-connector

import mysql.connector

db = mysql.connector.connect(
  host="34.94.61.132",
  user="root",
  passwd="superadmin123",
  database='inventory'
)

cursor = db.cursor()
## Fill out the Query to find products that have a retail cost greater than $6

result = cursor.fetchall()
print(result)